In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import copy
import random
from PIL import Image
import shutil
from urllib.request import urlretrieve
import os
import cv2
import time

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import torchvision
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2

import glob
from tqdm import tqdm
from tqdm.notebook import tqdm

from torchvision.datasets import OxfordIIITPet
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, ConcatDataset
from torchvision import transforms
import torchvision.transforms as tt

cudnn.benchmark = True

# Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [3]:
dataset = pd.read_csv(f"annotations/trainval.txt", sep=" ", 
                      names = ["Image", "ID", "SPECIES", "BREED ID"])
dataset.reset_index(drop=True)
dataset.head()

,Image,ID,SPECIES,BREED ID
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [4]:
#creating mappings between class labels and breeds
image_ids = []
labels = []
with open(f"annotations/trainval.txt") as file:
    for line in file:
        image_id, label, *_ = line.strip().split()
        image_ids.append(image_id)
        labels.append(int(label)-1)

classes = [
    " ".join(part.title() for part in raw_cls.split("_"))
    for raw_cls, _ in sorted(
        {(image_id.rsplit("_", 1)[0], label) for image_id, label in zip(image_ids, labels)},
        key=lambda image_id_and_label: image_id_and_label[1],
    )
    ]
idx_to_class = dict(zip(range(len(classes)), classes))
idx_to_class

{0: 'Abyssinian',
 1: 'American Bulldog',
 2: 'American Pit Bull Terrier',
 3: 'Basset Hound',
 4: 'Beagle',
 5: 'Bengal',
 6: 'Birman',
 7: 'Bombay',
 8: 'Boxer',
 9: 'British Shorthair',
 10: 'Chihuahua',
 11: 'Egyptian Mau',
 12: 'English Cocker Spaniel',
 13: 'English Setter',
 14: 'German Shorthaired',
 15: 'Great Pyrenees',
 16: 'Havanese',
 17: 'Japanese Chin',
 18: 'Keeshond',
 19: 'Leonberger',
 20: 'Maine Coon',
 21: 'Miniature Pinscher',
 22: 'Newfoundland',
 23: 'Persian',
 24: 'Pomeranian',
 25: 'Pug',
 26: 'Ragdoll',
 27: 'Russian Blue',
 28: 'Saint Bernard',
 29: 'Samoyed',
 30: 'Scottish Terrier',
 31: 'Shiba Inu',
 32: 'Siamese',
 33: 'Sphynx',
 34: 'Staffordshire Bull Terrier',
 35: 'Wheaten Terrier',
 36: 'Yorkshire Terrier'}

In [5]:
dataset['ID'] = dataset['ID'] - 1

decode_map = idx_to_class
def decode_label(label):
    return decode_map[int(label)]

dataset["class"] = dataset["ID"].apply(lambda x: decode_label(x))

dataset.head()


,Image,ID,SPECIES,BREED ID,class
0,Abyssinian_100,0,1,1,Abyssinian
1,Abyssinian_101,0,1,1,Abyssinian
2,Abyssinian_102,0,1,1,Abyssinian
3,Abyssinian_103,0,1,1,Abyssinian
4,Abyssinian_104,0,1,1,Abyssinian


In [6]:
dataset['class'].value_counts()

class
Abyssinian                    100
Russian Blue                  100
Maine Coon                    100
Miniature Pinscher            100
Persian                       100
Pomeranian                    100
Pug                           100
Ragdoll                       100
Saint Bernard                 100
American Bulldog              100
Samoyed                       100
Scottish Terrier              100
Shiba Inu                     100
Sphynx                        100
Staffordshire Bull Terrier    100
Wheaten Terrier               100
Leonberger                    100
Keeshond                      100
Japanese Chin                 100
Havanese                      100
American Pit Bull Terrier     100
Basset Hound                  100
Beagle                        100
Bengal                        100
Birman                        100
Boxer                         100
British Shorthair             100
Chihuahua                     100
English Setter                100
German S

In [7]:
from sklearn.model_selection import train_test_split

y = dataset['class']
x = dataset['Image']

trainval, x_test, y_trainval, y_test = train_test_split(x, y,
                                                        stratify=y, 
                                                        test_size=0.2,
                                                        random_state=42)

x_train, x_val, y_train, y_val = train_test_split(  trainval, y_trainval,
                                                    stratify=y_trainval, 
                                                    test_size=0.3,
                                                    random_state=42)

In [8]:
images_directory = os.path.join("images", "images")
masks_directory = os.path.join("annotations", "trimaps") #includ pixel values of a mask image

train_images_filenames = x_train.reset_index(drop=True)
val_images_filenames = x_val.reset_index(drop=True)
test_images_filenames = x_test.reset_index(drop=True)

In [18]:
from PIL import Image

class OxfordPetDataset(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory, transform=None, transform_mask=None):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames.loc[idx] + '.jpg' 
        image = Image.open(os.path.join(self.images_directory, image_filename)).convert('RGB')
        mask = Image.open(
            os.path.join(self.masks_directory, image_filename.replace(".jpg", ".png")))
        #mask = preprocess_mask(mask)
        if self.transform is not None:
            transformed = self.transform(image)
            transformed_m = self.transform_mask(mask)
            image = transformed
            mask = transformed_m
        return image, mask

In [19]:
#For Training Data
train_transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.ToTensor(), # [0, 255] -> [0.0, 1.0]
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #(pixel-mean)/std => [0.0, 1.0] ->  [-1.0, 1.0] 
#For Class Label Image: Label images are used in segmentation tasks to indicate the class of the object to which each pixel belongs.
target_transform = transforms.Compose([transforms.PILToTensor(),  #Maintains the original pixel value range  
                                       transforms.Resize((256, 256)),   
                                       # Start Class Label from 0, Dimention Reduction, Label Data Type Change  
                                       transforms.Lambda(lambda x: (x-1).squeeze().type(torch.LongTensor)) ])

In [20]:
train_dataset = OxfordPetDataset(train_images_filenames, 
                                 images_directory, 
                                 masks_directory, 
                                 transform=train_transform, 
                                 transform_mask=target_transform)


val_dataset = OxfordPetDataset(val_images_filenames,
                               images_directory,
                               masks_directory,
                               transform=train_transform,
                               transform_mask=target_transform)

In [23]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

def get_model(num_classes):
    # Load a pre-trained ResNet50 model with an FPN backbone.
    backbone = resnet_fpn_backbone('resnet50', pretrained=True)
    model = FasterRCNN(backbone, num_classes=num_classes)
    
    return model

# Assuming two classes: 1 for pet face + 1 for background.
num_classes = 2
model = get_model(num_classes)

c:\Users\alice\Documents\NLP\venv\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\alice\Documents\NLP\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alice\Documents\NLP\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ali

In [29]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import os

class PetFaceDataset(Dataset):
    def __init__(self, annotations_file, images_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        # Assuming img_path and bounding boxes are correctly loaded here
        img_path = os.path.join(self.images_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        
        # Load your bounding box annotations here
        box = self.img_labels.iloc[idx, 1:5].values
        box = torch.tensor(box, dtype=torch.float32)
        
        # Ensure labels are loaded or defined here, for simplicity assuming label is 1 for all
        labels = torch.tensor([1], dtype=torch.int64)  # Assuming all objects are 'pet face'

        target = {}
        target["boxes"] = box.unsqueeze(0)  # Model expects boxes in a list of tensors
        target["labels"] = labels

        if self.transform:
            image = self.transform(image)

        return image, target


# Transformation pipeline
transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


In [31]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def create_model(num_classes):
    # Load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

# Assuming 2 classes: 1 for pet face + 1 for background
model = create_model(num_classes=0)


c:\Users\alice\Documents\NLP\venv\Lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [28]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
from torch.optim.lr_scheduler import StepLR

# Assuming the dataset has been instantiated as `train_dataset`
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Move model to the right device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Parameters
num_epochs = 3
lr = 0.005

# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=lr, momentum=0.9, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Example scheduler

# Training Loop
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, targets in train_dataloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    avg_loss = running_loss / len(train_dataloader)
    print(f"Epoch: {epoch+1}, Average Loss: {avg_loss}")
    scheduler.step()  # Adjust learning rate

AttributeError: 'Tensor' object has no attribute 'items'

In [1]:
strs = ["flower","flow","flight"]
list_s = list(zip(*strs))
print(list_s)

[('f', 'f', 'f'), ('l', 'l', 'l'), ('o', 'o', 'i'), ('w', 'w', 'g')]


In [7]:
strs = ["flower","flow","flight"]
for i in zip(*strs):
    print(i)

('f', 'f', 'f')
('l', 'l', 'l')
('o', 'o', 'i')
('w', 'w', 'g')


In [11]:
s="()[]{}"
print(s[3])

]


In [18]:
num_id = ms_project['id'].nunique()

In [24]:
pd.sorted_values

)
